In [1]:
!pip install --upgrade scikit-learn
!pip install xgboost
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import joblib
import pickle
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 90.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 1.2 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded=files.upload()

Saving crop_yield.csv to crop_yield.csv


In [3]:
df=pd.read_csv("crop_yield.csv")
df.head()

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,West,Sandy,Cotton,897.077239,27.676966,False,True,Cloudy,122,6.555816
1,South,Clay,Rice,992.673282,18.026142,True,True,Rainy,140,8.527341
2,North,Loam,Barley,147.998025,29.794042,False,False,Sunny,106,1.127443
3,North,Sandy,Soybean,986.866331,16.644190,False,True,Rainy,146,6.517573
4,South,Silt,Wheat,730.379174,31.620687,True,True,Cloudy,110,7.248251


In [4]:
import pandas as pd
df = df.sample(frac=1, random_state=42)

In [5]:
categorical_cols = [
    "Fertilizer_Used",
    "Region",
    "Soil_Type",
    "Crop",
    "Irrigation_Used",
    "Weather_Condition"
]

numeric_cols = [
    "Rainfall_mm","Temperature_Celsius","Days_to_Harvest"
]
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])




In [6]:
X=df.drop("Yield_tons_per_hectare",axis=1)
y=df["Yield_tons_per_hectare"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest
987231,3,5,1,0.634488,-0.502602,0,0,2,0.597416
79954,1,0,1,1.195387,-0.614085,0,0,1,-1.020869
567130,1,4,0,0.970171,-0.482624,1,1,1,1.368028
500891,3,0,1,-1.332936,-1.469372,0,1,2,-0.327318
55399,0,5,3,-0.151832,-1.260568,0,1,0,-1.521767
...,...,...,...,...,...,...,...,...,...
259178,1,0,3,-1.343418,-0.078964,0,1,0,-0.096135
365838,1,3,1,0.195209,0.320586,1,0,2,-0.905277
131932,1,0,1,-0.363033,1.159542,0,1,1,-0.712624
671155,1,2,1,0.418834,-1.092037,1,1,0,-0.597033


In [7]:
model=LinearRegression()
model= model.fit(X_train, y_train)
preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds) ** 0.5
r2 = r2_score(y_test, preds)
mae=mean_absolute_error(y_test,preds)
rmse=np.sqrt(mse)
print("MAE: ",mae)
print("MSE: ",mse)
print("RMSE: ",rmse)
print("R2: ",r2)
joblib.dump(model,"lr_model.pkl")

MAE:  0.39941178536973754
MSE:  0.5004246332256785
RMSE:  0.7074069784965925
R2:  0.912992518793533


['lr_model.pkl']

In [9]:
model=XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        random_state=42
    )
model= model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = mean_squared_error(y_test, preds) ** 0.5
r2 = r2_score(y_test, preds)
mae=mean_absolute_error(y_test,preds)
rmse=np.sqrt(mse)
print("MAE: ",mae)
print("MSE: ",mse)
print("RMSE: ",rmse)
print("R2: ",r2)
joblib.dump(model,"xgb_model.pkl")

MAE:  0.3998328839873551
MSE:  0.5004246332256785
RMSE:  0.7074069784965925
R2:  0.9127878892735823


['xgb_model.pkl']

In [10]:
model= GradientBoostingRegressor(random_state=42)
model= model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = mean_squared_error(y_test, preds) ** 0.5
r2 = r2_score(y_test, preds)
mae=mean_absolute_error(y_test,preds)
rmse=np.sqrt(mse)
print("MAE: ",mae)
print("MSE: ",mse)
print("RMSE: ",rmse)
print("R2: ",r2)
joblib.dump(model,"gbr_model.pkl")

MAE:  0.4007517825960665
MSE:  0.5004246332256785
RMSE:  0.7074069784965925
R2:  0.9123864971789579


['gbr_model.pkl']

In [14]:
from google.colab import files
joblib.dump(scaler, "scaler.pkl")
joblib.dump(encoders, "label_encoders.pkl")

# Download trained models
files.download("lr_model.pkl")
files.download("rfr_model.pkl")
files.download("gbr_model.pkl")
files.download("xgb_model.pkl")

# Download preprocessing objects
files.download("scaler.pkl")
files.download("label_encoders.pkl")

print("✅ Training complete")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Training complete
